First is necessary to know which files are going to be used in the data set, in our case we tried to use all the information avaiable (so we should have 1012 patient id's inside our local copy of the LIDC-IDRI) so that we could get more accurate results and since we would be using "one" image from each pacient "nodule"

In [1]:
import os

#specify the path to your dataset where you have the folders with the name of the pacients
directory="data/dataset/LIDC-IDRI/"
#list all the folders inside our local copy of LIDC-IDRI (that correspond to the patient id's that are going to be used to get the scans and annotations)
folders=os.listdir(directory)

print(f"Number of patients in the dataset: {len(folders)}")
#just so that it is easier to know which patients we are don't have in our copy of the LIDC-IDRI
folders.sort()
print()
#printing the patients id's that we couldn't download
print("List of the id's that are missing from our dataset:")
count=1
for folder in folders:
    if(str(count) not in folder):
        print(folder)
        count=int(folder.split('-')[-1])
    count+=1


Number of patients in the dataset: 1010

List of the id's that are missing from our dataset:
LIDC-IDRI-0239
LIDC-IDRI-0586


As it is possible to see the number is diferent from the expected (1012 number of people that are avaiable in the original LODC-IRDI), this is due to the fact that during the download (which was made using the offical tool nbia-data-retriver) we did not have authority to download this instances

In [ ]:
import pylidc as pl

n_lesions=0
scans_0_nods=[]

for folder in folders:
    pid=folder

    #get the scann corresponding to the patient_id that we have 
    scan=pl.query(pl.Scan).filter(pl.Scan.patient_id==pid).first()
    #get all the nodules annotations, each list corresponds to the same nodules
    nods=scan.cluster_annotations()    
    n_lesions+=len(nods)
    if(len(nods)==0):
        scans_0_nods.append([scan.patient_id,scan.annotations])
        


print(f"Number of lesions: {n_lesions}" )
print(f"Number of scanns with 0 nodules: {len(scans_0_nods)}")
#print(scans_0_nods)
#13 of them gave an error so in total we have 123 scanns with 0 annotations


As it is possible to see above some of the scans don't have nodules/annotations 

In the following code we use VG16 to extract features from the images of the lesions

In [3]:
def get_malignancy(anns):
    #Get the Malignancy of the lesion
    #
    #   1-‘Highly Unlikely’
    #   2-‘Moderately Unlikely’
    #   3-‘Indeterminate’
    #   4-‘Moderately Suspicious’
    #   5-‘Highly Suspicious’
    #this feature will be used as a label to the dataset
    malignancy=0
    for i in range(len(anns)):
        malignancy+=anns[i].malignancy
    malignancy/=len(anns)
    malignancy=round(malignancy)

    return malignancy

In [11]:
def get_calcification(anns):
    #Get the pattern of calcification if present
    # 
    #   1-‘Popcorn’
    #   2-‘Laminated’
    #   3-‘Solid’
    #   4-‘Non-central’
    #   5-‘Central’
    #   6-‘Absent’
    calcification=0
    for i in range(len(anns)):
        calcification+=anns[i].calcification
    calcification/=len(anns)
    calcification=round(calcification)
    return calcification

In [12]:
def get_lobulation(anns):
    #Get the degree of lobulationof the lesion
    #
    #   1-‘No Lobulation’
    #   2-‘Nearly No Lobulation’
    #   3-‘Medium Lobulation’
    #   4-‘Near Marked Lobulation’
    #   5-‘Marked Lobulation’

    lobulation=0
    for i in range(len(anns)):
        lobulation+=anns[i].lobulation
    lobulation/=len(anns)
    lobulation=round(lobulation)
    return lobulation

In [13]:
def get_spiculation(anns):
    #Get the extent of spiculation present in the lesion
    #
    #   ‘No Spiculation’
    #   ‘Nearly No Spiculation’
    #   ‘Medium Spiculation’
    #   ‘Near Marked Spiculation’
    #   ‘Marked Spiculation’
    spiculation=0
    for i in range(len(anns)):
        spiculation+=anns[i].spiculation
    spiculation/=len(anns)
    spiculation=round(spiculation)
    return spiculation

In [14]:
def get_internal_texture(anns):
    #Get the internal texture of the lesion
    #
    #    1-‘Non-Solid/GGO’
    #    2-‘Non-Solid/Mixed’
    #    3-‘Part Solid/Mixed’
    #    4-‘Solid/Mixed’
    #    5-‘Solid’
    texture=0
    for i in range(len(anns)):
        texture+=anns[i].texture
    texture/=len(anns)
    texture=round(texture)
    return texture

In [15]:
def get_diameter_surface_area_volume(anns):
    #Get the mean value of all anotations for this lesion of the diameter, surface_area, volume
    diameter=0
    surface_area=0
    volume=0
    for i in range(len(anns)):
        diameter+=anns[i].diameter
        surface_area+=anns[i].surface_area
        volume+=anns[i].volume
    diameter/=len(anns)
    surface_area/=len(anns)
    volume/=len(anns)

    return diameter,surface_area,volume

In [16]:
import numpy as np

def get_pixeis_in(cmask):
    #Calculating the number of pixeis inside a lesion
    return np.sum(cmask)

In [26]:
from skimage.filters import sobel

def get_circularity(cmask):
    # Calculate circularity
    # the closer to 1 the closer to a perfect circle
    circularity=0
    for i in range(cmask.shape[-1]):
        area_k_slice_pixels=np.sum(cmask[:,:,i])#this way we get the number of pixeis of the k
        # Calculating the perimeter(in pixels) by appling an edge filter to the mask
        edges=sobel(cmask)
        perimeter_piexels=np.sum(edges[:,:,i])#so that we get the perimeter of the slice i of the lesion
        if(perimeter_piexels==0):
            circularity+=0
        else:
            circularity += (4 * np.pi * area_k_slice_pixels) / (perimeter_piexels ** 2)
    circularity/=cmask.shape[-1]

    return circularity

In [18]:
def get_median_hu(nodule_hu_values):
    #Calculate the median HU inside of the lesion
    #vol[ccbox][cmask] represents the HU inside the lesion
    return np.median(nodule_hu_values)

In [19]:
def get_std_hu(nodule_hu_values):
    #Calculate the the standard variation of HU inside the lesion
    return np.std(nodule_hu_values)

In [20]:
def get_var_hu(nodule_hu_values):
    #Calculate the variance of HU inside the lesion
    return np.var(nodule_hu_values)

In [21]:
def get_mean_hu(nodule_hu_values):
    #Calculate the mean of HU inside the lesion
    return np.mean(nodule_hu_values)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from pylidc.utils import consensus
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

#load pre-trained model withput the top layers
base_model=VGG16(weights='imagenet',include_top=False, input_shape=(512,512,3))

#add global pooling to the method
x=base_model.output
x=GlobalAveragePooling2D()(x)
cnn_model=Model(inputs=base_model.input,outputs=x)


#create the DataFrame where we ccn features are going to be stored

#create the columns names
columns_name=["PatientID",
              "Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion"
              ]
for i in range(512):
    columns_name.append("cnn_feature "+str(i))

columns_name.append("Label")

df=pd.DataFrame(columns=columns_name)


for folder in folders:
    
    pid=folder
    print(f"Currently in scan: {pid}")

    #get the scann corresponding to the patient_id that we have 
    scan=pl.query(pl.Scan).filter(pl.Scan.patient_id==pid).first()
    
    #get all the nodules annotations, each list corresponds to the same lesions 
    nods=scan.cluster_annotations()
    #nods is a list of list of annotations agrouped by nodule

    #create an volume with all of the dicom images of that scan (basicly an 3D image of the CT)
    vol=scan.to_volume()

    padding=[(30,20),(10,25),(0,0)]
    #anns represents all the annotations for one lesion in this scann
    #nods represents a list of all the annotations for each lesion in this scan 
    for anns in nods:
        #get the concensus for the contours of this nodule
        cmask,cbbox,masks=consensus(anns,clevel=0.5,pad=padding)

        #get the central slice of the computed bounding box
        k=int(0.5*(cbbox[2].stop-cbbox[2].start))

        img=vol[cbbox][:,:,k]


        #applying possible usefull filters since we don't have that many lesions (2625)
        from skimage.filters.rank import entropy
        from skimage.morphology import disk
        from skimage.util import img_as_ubyte
        entropy_img=entropy(img_as_ubyte(img),disk(1))


        from skimage.filters import sobel
        sobel_img=sobel(img)

        #getting the images ready for the cnn
        imgs=[img,entropy_img,sobel_img]

        for i in range(1):#we can change this after if we want to use the filters
            #making each image have 3 dimensions
            imgs[i] = np.expand_dims(imgs[i], axis=-1)

            #resizing the images so that they match the input of the cnn
            imgs[i]=tf.image.resize(imgs[i],(512,512))

            #converting the images to rgb
            if(imgs[i].shape[-1]==1):
                imgs[i]=tf.image.grayscale_to_rgb(imgs[i])
            
            #preprocess to the VGG16
            imgs[i]=preprocess_input(imgs[i].numpy())

            #get the features learned by the cnn
            features=cnn_model(np.expand_dims(imgs[i], axis=0))
            features=features.numpy().flatten()

            #create the row with everything that is going in the dataset 
            row=[pid]
            #all of this features are explainded in extract_features.ipynb
            row.append(get_calcification(anns))
            row.append(get_lobulation(anns))
            row.append(get_spiculation(anns))
            row.append(get_internal_texture(anns))
            row.extend(get_diameter_surface_area_volume(anns))
            row.append(get_pixeis_in(cmask))
            row.append(get_circularity(cmask))
            row.append(get_median_hu(vol[cbbox][cmask]))
            row.append(get_std_hu(vol[cbbox][cmask]))
            row.append(get_var_hu(vol[cbbox][cmask]))
            row.append(get_mean_hu(vol[cbbox][cmask]))
            #this are the cnn_extracted features
            for x in features:
                row.append(x)
            #this is the label
            row.append(get_malignancy(anns))
            df.loc[len(df.index)]=row
        
    #     break #this breaks are for when debugging not having to run all the scanns
    # break #this breaks are for when debugging not having to run all the scanns


In [4]:
import pandas as pd

df=pd.read_csv('cnn+pylidc_features.csv')
print(df.shape)
df.head()

(2625, 527)


,PatientID,Calcification,Lobulation,Spiculation,Internal Texture,Diameter (mm),Surface Area (mm^2),Volume (mm^3),Pixeis_in_lesion,Circularity,...,cnn_feature 503,cnn_feature 504,cnn_feature 505,cnn_feature 506,cnn_feature 507,cnn_feature 508,cnn_feature 509,cnn_feature 510,cnn_feature 511,Label
0,LIDC-IDRI-0001,6,3,4,5,32.755812,2491.466573,6989.673615,5428,0.121212,...,0.080818,1.799512,0.474278,6.975934,1.273983,4.503269,0.129298,9.106273,0.563464,5
1,LIDC-IDRI-0002,6,1,1,2,30.781671,2807.198994,7244.667508,14252,0.243756,...,3.473532,0.753911,0.949201,1.776874,2.387758,4.194271,1.476390,5.335127,1.748041,4
2,LIDC-IDRI-0003,6,1,1,1,31.664468,1996.252117,4731.410934,2542,0.154584,...,0.801210,0.239504,9.201927,1.946184,5.459285,0.320320,0.616315,3.036831,0.429381,2
3,LIDC-IDRI-0003,6,2,3,4,31.001964,2225.677350,6519.463698,3241,0.139765,...,0.951502,1.994685,0.370287,5.199282,1.392501,1.789750,0.671943,5.783095,0.068551,4
4,LIDC-IDRI-0003,6,2,2,5,13.309155,321.183599,472.089669,261,0.271896,...,0.095775,1.139045,0.818399,4.912726,11.126127,1.109776,1.508004,11.945731,0.484690,3


Store the current cnn features values to make it possible to make use them without having to calculate them again as it can take a while (93 min first time we tried)


In [29]:
df.to_csv('cnn+pylidc_features.csv', index=False)

In order to make the randomforest algorithm to work better we decided to make the number of features smaller (200)

In [5]:
from sklearn.decomposition import PCA
pca=PCA(n_components=200)
features_columns=df.loc[:,~df.columns.isin(['PatientID','Label',"Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion"])]
reduced_features=pca.fit_transform(features_columns)
reduced_features.shape

(2625, 200)

Formating the reduced features into an DataFrame

In [6]:
columns_name=[]
for i in range(200):
    columns_name.append("cnn_feature "+str(i))
    
reduced_df=pd.DataFrame(columns=columns_name, data=reduced_features)
reduced_df=pd.concat([df[['PatientID','Label',"Calcification",
              "Lobulation",
              "Spiculation",
              "Internal Texture",
              "Diameter (mm)",
              "Surface Area (mm^2)",
              "Volume (mm^3)",
              "Pixeis_in_lesion",
              "Circularity",
              "Median_HU_in_lesion",
              "STD_HU_in_lesion",
              "VAR_HU_in_lession",
              "Mean_HU_in_lesion"]],reduced_df,df['Label']], axis=1)
reduced_df.head()




,PatientID,Label,Calcification,Lobulation,Spiculation,Internal Texture,Diameter (mm),Surface Area (mm^2),Volume (mm^3),Pixeis_in_lesion,...,cnn_feature 191,cnn_feature 192,cnn_feature 193,cnn_feature 194,cnn_feature 195,cnn_feature 196,cnn_feature 197,cnn_feature 198,cnn_feature 199,Label
0,LIDC-IDRI-0001,5,6,3,4,5,32.755812,2491.466573,6989.673615,5428,...,0.103554,-0.673255,0.911037,0.195182,1.503029,2.497326,-1.285972,-0.113949,-0.123039,5
1,LIDC-IDRI-0002,4,6,1,1,2,30.781671,2807.198994,7244.667508,14252,...,-0.465596,0.751876,-1.025916,-0.876682,-0.427128,-1.996738,2.039433,-0.173065,3.660489,4
2,LIDC-IDRI-0003,2,6,1,1,1,31.664468,1996.252117,4731.410934,2542,...,2.096471,1.056720,0.507382,1.195086,-1.089219,0.402015,-1.525909,-0.753988,2.830438,2
3,LIDC-IDRI-0003,4,6,2,3,4,31.001964,2225.677350,6519.463698,3241,...,1.189942,-0.748756,3.139558,1.495741,1.706667,-0.051177,-1.470173,-0.269185,-0.570582,4
4,LIDC-IDRI-0003,3,6,2,2,5,13.309155,321.183599,472.089669,261,...,0.260855,-1.168901,2.171054,-1.587639,0.945781,0.030748,-0.701231,-2.497634,-0.823811,3


In [7]:
#storing the reduced cnn_features
reduced_df.to_csv('cnn_reduced+pylidc_features.csv',index=False)